## Tarea 4 (4.4)
Veinte números aleatorios se extraen de una distribución uniforme entre $0$ y $1$.

- Use monte carlo para estimar la probabilidad de que la suma de los veinte números esté entre $9$ y $10$. Ídem entre $15$ y $16$.
- Use el teorema central del límite para estimar la misma probabilidad.

Recuerde estimar el error de la simulación así como cuantificar con errores relativos la diferencia entre la simulación y la gaussiana.

Comente sobre su resultado.

Pista:

$$
\int_0^a dx\,e^{-x^2} = \frac{\sqrt{\pi}}{2}\text{erf}(a)\,,
$$

donde $\text{erf}$ es la una función especial llamada la función de error. Esta función se puede llamar usando el módulo `scipy.special`.


In [2]:
import numpy as np
from scipy.special import erf

In [8]:
N_simulaciones = 100000 #Cantidad de simulaciones
N_numeros = 20 #Cantidad de numeros aleatorios en cada simulacion

#Monte Carlo, genera de N_simulaciones sumas de 20 numeros aleatorios uniformes estre 0 y 1
random_sums = np.sum(np.random.uniform(0, 1, (N_simulaciones, N_numeros)), axis=1)

#Intervalos
Intervalo_9_10 = (9,10)
Intervalo_15_16 = (15, 16)

#Probabolidad con Monte Carlo
prob_mc_9_10 = np.mean((random_sums >= Intervalo_9_10[0]) & (random_sums <= Intervalo_9_10[1]))
prob_mc_15_16 = np.mean((random_sums >= Intervalo_15_16[0]) & (random_sums <= Intervalo_15_16[1]))

#Estimacion error en simulacion MC
error_mc_9_10 = np.sqrt(prob_mc_9_10 * (1 - prob_mc_9_10) / N_simulaciones)
error_mc_15_16 = np.sqrt(prob_mc_15_16 * (1 - prob_mc_15_16) / N_simulaciones)

#Teo central del limite
mu = N_numeros * 0.5  # Media de la suma
sigma = np.sqrt(N_numeros / 12)  # Desviación estándar de la suma


#Probabilidad usando aprox normal(tcl = teorema central del limite)
def prob_tcl(a, b, mu, sigma):
  return 0.5 * (erf((b - mu) / (sigma * np.sqrt(2))) - erf((a - mu) / (sigma * np.sqrt(2))))


#Prob usando TCL
prob_tcl_9_10 = prob_tcl(9, 10, mu, sigma)
prob_tcl_15_16 = prob_tcl(15, 16, mu, sigma)

#Eror entre MC y TCL
error_rel_9_10 = abs(prob_mc_9_10 - prob_tcl_9_10) / prob_tcl_9_10
error_rel_15_16 = abs(prob_mc_15_16 - prob_tcl_15_16) / prob_tcl_15_16




In [9]:
# Resultados
print(f"Intervalo [9, 10]:")
print(f"Probabilidad Monte Carlo: {prob_mc_9_10:.5f} ± {error_mc_9_10:.5f}")
print(f"Probabilidad TCL: {prob_tcl_9_10:.5f}")
print(f"Error relativo: {error_rel_9_10*100:.2f}%\n")

print(f"Intervalo [15, 16]:")
print(f"Probabilidad Monte Carlo: {prob_mc_15_16:.5f} ± {error_mc_15_16:.5f}")
print(f"Probabilidad TCL: {prob_tcl_15_16:.5f}")
print(f"Error relativo: {error_rel_15_16*100:.2f}%")


Intervalo [9, 10]:
Probabilidad Monte Carlo: 0.27875 ± 0.00142
Probabilidad TCL: 0.28071
Error relativo: 0.70%

Intervalo [15, 16]:
Probabilidad Monte Carlo: 0.00001 ± 0.00001
Probabilidad TCL: 0.00005
Error relativo: 80.80%


# Nota: 50

In [3]:
points= 160000
sims = 10

def prob_intervalo(a, b, res):
    return np.count_nonzero((res > a)*(res < b))/ points

probs1 = np.zeros(sims)
probs2 = np.zeros(sims)

for i in range(sims):
    res = np.array([sum(np.random.random(10)) for i in range(points)])
    probs1[i] = prob_intervalo(4, 5, res)
    probs2[i] = prob_intervalo(8, 9, res)

In [4]:
prob1mc = probs1[0]

In [5]:
prob1mc

0.364575

In [6]:
prob2mc = probs2[0]

In [7]:
prob2mc

0.0002875

In [8]:
probs1.std()

0.0012949149344648053

In [9]:
probs2.std()

4.833557825246327e-05

In [10]:
import scipy as sp

La varianza de una distribución uniforme es

$$
\int_0^1 dx\, (x - 1/2)^2 = \frac{1}{12}
$$

tal que la gaussiana del teorema central del límite tiene una varianza

$$
\sigma^2 = \frac{10}{12}
$$

In [11]:
sigma = np.sqrt(10/12)
mu = 10/2

La probabilidad de obtener un resultado entre $a$ y $b$ para la gaussiana se puede obtener a partir de la función de error 

$$
P = \frac{1}{2}\left(\text{erf}\left(\frac{b - \mu}{\sqrt{2}\sigma}\right) - \text{erf}\left(\frac{a - \mu}{\sqrt{2}\sigma}\right)\right)
$$

In [12]:
def prob_intervalo_gauss(a, b):
    p1 = sp.special.erf((b - mu)/(np.sqrt(2)*sigma))
    p2 = sp.special.erf((a - mu)/(np.sqrt(2)*sigma))
    return (p1 - p2)/2

In [13]:
prob1g = prob_intervalo_gauss(4, 5)

In [14]:
prob2g = prob_intervalo_gauss(8, 9 )

In [15]:
np.abs(prob1mc - prob1g)/prob1g

0.0034013375911500264

In [16]:
np.abs(prob2mc - prob2g)/prob2g

0.42685104645442656

In [17]:
probs2.std()/prob2g

0.09635995163276348

Vemos que el error sobre la probabilidad en el segundo caso es grande. Este es un error de la aproximación y no numérico ya que el error de amonte Carlo es mucho menor.
Esto ilustra el hecho que la convergencia a una gaussiana garantizada por el teorema central del límite es lenta en las colas, es decir a varias desviaciones estándar de la media.